In [1]:
def get_input(fname="test.txt"):
    input = []
    with open(fname) as f:
        for l in f.readlines():
            line = l.rstrip()
            input.append(list(line))
    return input

In [2]:
test_input = get_input("test.txt")
my_input = get_input("input.txt")

In [3]:
test_input

[['.', '.', 'F', '7', '.'],
 ['.', 'F', 'J', '|', '.'],
 ['S', 'J', '.', 'L', '7'],
 ['|', 'F', '-', '-', 'J'],
 ['L', 'J', '.', '.', '.']]

In [4]:
def print_grid(input):
    print('\n'.join(''.join(l) for l in input))

In [5]:
print_grid(test_input)

..F7.
.FJ|.
SJ.L7
|F--J
LJ...


In [6]:
from collections import deque

In [7]:
def print_distance_grid(g):
    print('\n'.join(''.join(str(i) if i >= 0 else '.' for i in l) for l in g))

In [8]:
NORTH, SOUTH, EAST, WEST = (-1, 0), (1, 0), (0, 1), (0, -1)
SHAPES = { 'L': (NORTH, EAST), 'J': (NORTH, WEST), 'F': (SOUTH, EAST), '7': (SOUTH, WEST), '|': (NORTH, SOUTH), '-': (EAST, WEST) }

In [9]:
def distances(o_input):
    start_position = None
    input = [list('.' * (len(o_input[0]) + 2))]
    input += [['.'] + l + ['.'] for l in o_input]
    input += [list('.' * (len(o_input[0]) + 2))]
    start_position = None
    start_shape = None
    for i, l in enumerate(input):
        for j, c in enumerate(l):
            if c == 'S':
                start_position = (i, j)
                reachable = set([NORTH, SOUTH, EAST, WEST])
                for direction in (NORTH, SOUTH, EAST, WEST):
                    shape = input[i + direction[0]][j + direction[1]]
                    if shape == '.':
                        reachable.remove(direction)
                    # blocking to the right
                    if shape in { '|', 'L', 'F' } and direction == EAST:
                        reachable.remove(direction)
                    # blocking to the left
                    if shape in { '|', 'J', '7' } and direction == WEST:
                        reachable.remove(direction)
                    # blocking to the top
                    if shape in { '-', 'L', 'J' } and direction == NORTH:
                        reachable.remove(direction)
                    # blocking to the bottom
                    if shape in { '-', 'F', '7' } and direction == SOUTH:
                        reachable.remove(direction)
                for shape, d in SHAPES.items():
                    if reachable == set(d):
                        input[i][j] = shape
                        break
    distances = [[-1 for _ in l] for l in input]
    coordinates = { start_position }
    dist = 0
    distances[start_position[0]][start_position[1]] = 0
    while coordinates:
        dist += 1
        new_coordinates = set()
        for c in coordinates:
            shape = input[c[0]][c[1]]
            for d in SHAPES[shape]:
                i, j = c[0] + d[0], c[1] + d[1]
                if distances[i][j] >= 0:
                    continue
                distances[i][j] = dist
                new_coordinates.add((i, j))
        coordinates = new_coordinates
    return dist - 1

In [10]:
distances(test_input)

8

In [11]:
distances(my_input)

7173

In [12]:
from collections import deque

In [13]:
# clockwise
NEXT_DIRECTION = {
    'L': {
        SOUTH: EAST,
        WEST: NORTH
    },
    'J': {
        SOUTH: WEST,
        EAST: NORTH
    },
    'F': {
        NORTH: EAST,
        WEST: SOUTH
    },
    '7': {
        EAST: SOUTH,
        NORTH: WEST
    },
}

INSIDE = {
    'L': {
        SOUTH: (0, -1),
        WEST: (-1, 1)
    },
    'J': {
        SOUTH: (-1, -1),
        EAST: (1, 0)
    },
    'F': {
        NORTH: (1, 1),
        WEST: (0, -1)
    },
    '7': {
        NORTH: (0, 1),
        EAST: (1, -1)
    },
    '|': {
        NORTH: (0, 1),
        SOUTH: (0, -1)
    },
    '-': {
        EAST: (1, 0),
        WEST: (-1, 0)
    }
}

def enclosed(o_input, start_direction=NORTH):
    start_position = None
    input = [list('.' * (len(o_input[0]) + 2))]
    input += [['.'] + l + ['.'] for l in o_input]
    input += [list('.' * (len(o_input[0]) + 2))]
    start_position = None
    start_shape = None
    for i, l in enumerate(input):
        for j, c in enumerate(l):
            if c == 'S':
                start_position = (i, j)
                reachable = set([NORTH, SOUTH, EAST, WEST])
                for direction in (NORTH, SOUTH, EAST, WEST):
                    shape = input[i + direction[0]][j + direction[1]]
                    if shape == '.':
                        reachable.remove(direction)
                    # blocking to the right
                    if shape in { '|', 'L', 'F' } and direction == EAST:
                        reachable.remove(direction)
                    # blocking to the left
                    if shape in { '|', 'J', '7' } and direction == WEST:
                        reachable.remove(direction)
                    # blocking to the top
                    if shape in { '-', 'L', 'J' } and direction == NORTH:
                        reachable.remove(direction)
                    # blocking to the bottom
                    if shape in { '-', 'F', '7' } and direction == SOUTH:
                        reachable.remove(direction)
                for shape, d in SHAPES.items():
                    if reachable == set(d):
                        input[i][j] = shape
                        break
    edges = [start_position]
    nxt_pos = None
    enclosed_coords = set()
    visited_edges = { start_position }
    direction = start_direction
    while nxt_pos != start_position:
        nxt_pos = edges[-1][0] + direction[0], edges[-1][1] + direction[1]
        c = input[nxt_pos[0]][nxt_pos[1]]
        if direction in INSIDE[c]:
            enclosed_coords.add((nxt_pos[0] + INSIDE[c][direction][0], nxt_pos[1] + INSIDE[c][direction][1]))
        if c in NEXT_DIRECTION:
            direction = NEXT_DIRECTION[c][direction]
        if nxt_pos != start_position:
            edges.append(nxt_pos)
            visited_edges.add(nxt_pos)
    enclosed_coords -= visited_edges
    q = deque(enclosed_coords)
    while len(q):
        pos = q.pop()
        for delta in ((-1, 0), (1, 0), (0, -1), (0, 1)):
            p = (pos[0] + delta[0], pos[1] + delta[1])
            if p not in enclosed_coords and p not in edges:
                enclosed_coords.add(p)
                q.append(p)
    for i, j in edges:
        input[i][j] = '*'
    for i, j in enclosed_coords:
        input[i][j] = 'I'
    return len(enclosed_coords)

In [14]:
test2_input = get_input('test2.txt')

In [15]:
enclosed(test2_input, SOUTH)

10

In [16]:
enclosed(my_input, SOUTH)

291